## Machine Learning 프로젝트 수행을 위한 코드 구조화 

- ML project를 위해서 사용하는 템플릿 코드를 만듭니다.

1. **필요한 라이브러리와 데이터를 불러옵니다.**


2. **EDA를 수행합니다.** 이 때 EDA의 목적은 풀어야하는 문제를 위해서 수행됩니다.


3. **전처리를 수행합니다.** 이 때 중요한건 **feature engineering**을 어떻게 하느냐 입니다.


4. **데이터 분할을 합니다.** 이 때 train data와 test data 간의 분포 차이가 없는지 확인합니다.


5. **학습을 진행합니다.** 어떤 모델을 사용하여 학습할지 정합니다. 성능이 잘 나오는 GBM을 추천합니다.


6. **hyper-parameter tuning을 수행합니다.** 원하는 목표 성능이 나올 때 까지 진행합니다. 검증 단계를 통해 지속적으로 **overfitting이 되지 않게 주의**하세요.


7. **최종 테스트를 진행합니다.** 데이터 분석 대회 포맷에 맞는 submission 파일을 만들어서 성능을 확인해보세요.

## 1. 라이브러리, 데이터 불러오기

In [ ]:
# 데이터분석 4종 세트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 모델들, 성능 평가
# (저는 일반적으로 정형데이터로 머신러닝 분석할 때는 이 2개 모델은 그냥 돌려봅니다. 특히 RF가 테스트하기 좋습니다.)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm.sklearn import LGBMRegressor

# 상관관계 분석, VIF : 다중공선성 제거
from statsmodels.stats.outliers_influence import variance_inflation_factor

# KFold(CV), partial : optuna를 사용하기 위함
from sklearn.model_selection import KFold
from functools import partial

# hyper-parameter tuning을 위한 라이브러리, optuna
import optuna

In [ ]:
# flag setting
feature_reducing = True

In [ ]:
# 데이터를 불러옵니다.
train = pd.read_csv("../input/mercedes-benz-greener-manufacturing/train.csv.zip")
test = pd.read_csv('../input/mercedes-benz-greener-manufacturing/test.csv.zip')

## 2. EDA

- 데이터에서 찾아야 하는 기초적인 내용들을 확인합니다.


- class imbalance, target distribution, outlier, correlation을 확인합니다.

In [ ]:
train

In [ ]:
## On your Own
# 1. 데이터 크기 확인
train.info() # 4209 x 378

In [ ]:

# 2. 결측치 체크 : column 방향으로 먼저 체크.
train.columns[train.isnull().any()]  # 결측치 없음.
train[train.isnull().any(axis=1)]  # 결측치 없음.

In [ ]:
train

In [ ]:
# 3. dtype이 object인 column 체크. : categorical feature(가 될 가능성이 높은) 체크.
cat_features = train.columns[train.dtypes == 'object']  # dtype이 object인 Column을 뽑음.
train.loc[:, train.dtypes == 'object']   # dtype이 object인 모든 데이터를 뽑음.

train.columns[10:] # 앞에 10개를 제외한 나머지 모든 column
# 전체 column에서 "ID", "y", cat_features를 제외한 나머지 column
binary_features = list(set(train.columns) - set(cat_features) - {"ID", "y"})
#binary_features = np.setdiff1d(train.columns, cat_features)

# 순서를 안바꾸고, train.columns에서 ID, y, cat_feature를 제외한 나머지를 filtering하고 싶음.
# 3-1. ID, y, cat_features를 하나로 합침.
arr = np.hstack([["ID", "y"], cat_features])

# 3-2. 위에서 만든 column들에 포함이 안되면이라는 조건 생성.
mask = ~train.columns.isin(arr)

# 3-3. masking
binary_features = train.columns[mask]

In [ ]:
# 4. target distribution : target value의 분포 체크.
plt.figure(figsize=(12, 6))
sns.histplot(data=train, x="y")
plt.show()

In [ ]:
# 5. 상관관계 분석 : correlation matrix + heatmap
corr = train.drop(columns="ID").corr()
plt.figure(figsize=(24, 24))
sns.heatmap(data=corr, cmap="Blues")
plt.show()

이런 식으로 여러가지 그래프를 그려가며, 데이터에 대한 인사이트를 얻습니다!

In [ ]:
## TO-DO - EDA, 이전 코드 복습!

for col in cat_features:
    plt.figure(figsize=(12, 8))
    sns.countplot(data=train, x=col)
    plt.show()

In [ ]:
## binary features들 중에서 모든 값이 동일한 feature는 학습에 도움이 되지 않기 때문에, 제거합니다.
drop_cols = []
for col in binary_features:
    if train[col].nunique() == 1: # 해당 column에 전부 다 0이거나 1인 column들.
        print(col)
        drop_cols.append(col)

In [ ]:
# binary_features에서 column별 합이 0이거나 전체 길이(4209)인 column들을 추출합니다.
binary_features[(train[binary_features].sum() == 0) | (train[binary_features].sum() == len(train))]

### 3. 전처리

#### 불필요한 column 처리

In [ ]:
train = train.drop(columns=["ID"] + drop_cols)
test = test.drop(columns=["ID"] + drop_cols)
print(train.shape, test.shape)

#### Categorical Feature encoding

- Ordinal Encoding VS One-Hot encoding


- pandas VS sklearn

In [ ]:
# pandas로 ordinal encoding
encoding = "ODE"

# train과 test 데이터를 한번에 처리하기 위해서, 두 데이터를 합쳐서 전처리를 수행합니다. (모든 category에 대한 학습이 필요함.)
total = pd.concat([train, test])

if encoding == 'ODE':
    for f in cat_features:
        total[f] = pd.factorize(total[f])[0]

# pandas로 one-hot encoding
elif encoding == 'OHE':
    total = pd.get_dummies(data=total, columns=cat_features)
    
train = total[:len(train)]
test = total[len(train):].drop(columns='y')

In [ ]:
train

In [ ]:
test

#### 다중공선성 처리

In [ ]:
# 중복정보가 있는 column 제거하기 위해 상관계수를 확인해봅니다.


corr = train.drop(columns='y').corr()
corr

# X0 <--> X2, X12, X356
# X1 <--> ...
# X2 (skip)
# X3 
...
# X385 <--> ...

In [ ]:
threshold = 0.7
except_cols = []  # 이전에 이미 체크한 column들. (제거 대상으로 안봐도 되는 column들.)
remove_cols = []  # 상관관계가 threshold를 넘어서 제거할 column들.

for col in corr:
    if col in except_cols:  # 이미 체크를 한 column이라 지나갑니다.
        continue
    except_cols.append(col) # 같은 Column은 무조건 1이라서, 제외.
    row = np.abs(corr.loc[col]) # correlation matrix의 row 하나
    condition1 = row >= threshold  # 상관계수를 넘기는 column들.
    condition2 = ~corr.columns.isin(except_cols) # 이전에 이미 봤기 때문에 넘어가도 되는 column.
    temp = row[condition1 & condition2].index
    except_cols = except_cols + list(temp)  # threshold를 넘긴 column들이라서 제외.
    remove_cols = remove_cols + list(temp)  # 이전에 등장하지 않았던 새로운 Column들.

In [ ]:
len(remove_cols)

In [ ]:
# VIF(Variance Inflation Factor)를 이용하여 다중공선성(서로 상관이 높은) column들을 제거합니다.
# VIF가 1이라면, 다른 feature와 전혀 상관관계가 없고 그 때의 R^2는 0입니다.

## 여기서는 Ordinal encoding이 된 X0 ~ X8까지만 체크합니다.

#vifs = [variance_inflation_factor(train.drop(columns='y'), idx) for idx in range(len(train.columns)-1)]
vifs = [variance_inflation_factor(train.drop(columns='y'), idx) for idx in range(8)]

#vif_df = pd.DataFrame({"Features" : train.drop(columns='y').columns, "VIF" : vifs})
vif_df = pd.DataFrame({"Features" : train.drop(columns='y').columns[:8], "VIF" : vifs})
vif_df.sort_values(by="VIF", ascending=False)

In [ ]:
# VIF가 threshold를 넘기는 feature들을 제거합니다.
threshold = 10 # <-> R2 score = 0.9
vif_df[vif_df.VIF >= threshold].Features.values
# skip.

#### feature extraction

- 차원의 저주를 해결하거나, 데이터의 feature 조합을 이용하는 새로운 feature를 생성할 때, PCA를 사용합니다.

- 분석에 사용할 feature를 선택하는 과정도 포함합니다.

In [ ]:
# PCA 적용
if feature_reducing:
    from sklearn.decomposition import PCA
    
    #pca = PCA(n_components=15)   # 15차원으로 변환
    pca = PCA(n_components=0.95)  # 원본 데이터의 정보를 90% 보존하는 차원으로 변환
    train_pca = pca.fit_transform(train.drop(columns='y'))
    print(train_pca)

In [ ]:
pca_columns = [f"PC{i+1}" for i in range(train_pca.shape[1])]
pca_df = pd.DataFrame(data=train_pca, columns=pca_columns)
pca_df

### 4. 학습 데이터 분할

In [ ]:
# 첫번째 테스트용으로 사용하고, 실제 학습시에는 K-Fold CV를 사용합니다.
from sklearn.model_selection import train_test_split

if feature_reducing:
    X = pca_df
else:
    X = train.drop(columns=['y'] + remove_cols)  # 학습에 사용하지 않을 column들을 drop.
y = train.y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0xC0FFEE)

In [ ]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

### 5. 학습 및 평가

In [ ]:
# 간단하게 LightGBM 테스트
# 적당한 hyper-parameter 조합을 두었습니다. (항상 best는 아닙니다. 예시입니다.)

param_grid = {
    "max_bin" : 20,
    "learning_rate" : 0.0025,
    "objective" : "regression",
    "boosting_type" : "gbdt",
    "metric" : "mae",
    "sub_feature" : 0.345,
    "bagging_fraction" : 0.85,
    "bagging_freq" : 40,
    "num_leaves" : 512,
    "min_data" : 500,
    "min_hessian" : 0.05,
    "verbose" : 2,
    "feature_fraction_seed" : 2,
    "bagging_seed" : 3
}

model = LGBMRegressor(**param_grid)

In [ ]:
print("\nFitting LightGBM...")
model.fit(X_train, y_train)

In [ ]:
# metric은 그때마다 맞게 바꿔줘야 합니다.
from sklearn.metrics import r2_score
#from sklearn.metrics import mean_squared_error

evaluation_metric = r2_score

In [ ]:
print("Prediction")
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)


train_score = evaluation_metric(y_train, pred_train)
test_score = evaluation_metric(y_test, pred_test)

print("Train Score : %.4f" % train_score)
print("Test Score : %.4f" % test_score)

### 6. Hyper-parameter Tuning

> GridSearchCV

** LightGBM의 hyperparameter **

[Official Documentation] https://lightgbm.readthedocs.io/en/latest/Parameters-Tuning.html 

[Blog 1] https://smecsm.tistory.com/133

[Blog 2] https://towardsdatascience.com/kagglers-guide-to-lightgbm-hyperparameter-tuning-with-optuna-in-2021-ed048d9838b5

[Blog 3] https://nurilee.com/2020/04/03/lightgbm-definition-parameter-tuning/

In [ ]:
# GridSearchCV를 이용하여 가장 좋은 성능을 가지는 모델을 찾아봅시다. (이것은 첫번째엔 선택입니다.)
# Lightgbm은 hyper-parameter의 영향을 많이 받기 때문에, 저는 보통 맨처음에 한번 정도는 가볍게 GCV를 해봅니다.
# 성능 향상이 별로 없다면, lightgbm으로 돌린 대략적인 성능이 이 정도라고 생각하면 됩니다.
# 만약 성능 향상이 크다면, 지금 데이터는 hyper-parameter tuning을 빡빡하게 하면 성능 향상이 많이 이끌어 낼 수 있습니다.

from sklearn.model_selection import GridSearchCV

param_grid = {
    "max_depth" : [8, 16, None],
    "n_estimators" : [100, 300, 500],
    "max_bin" : [20],
    "learning_rate" : [0.001, 0.0025, 0.003],
    "objective" : ["regression"],
    "boosting_type" : ["gbdt"],
    "metric" : ["mae"],
    "sub_feature" : [0.345],
    "bagging_fraction" : [0.7, 0.75, 0.85],
    "bagging_freq" : [40],
    "num_leaves" : [256, 512],
    "min_data" : [500],
    "verbose" : [-1], # 필수
    "min_hessian" : [0.05],
    "feature_fraction_seed" : [2],
    "bagging_seed" : [3]
}  # 3 x 3 x 3 x 3 x 2 = 162 ---> GridSearch가 탐색하는 Hyper-parameter 조합 수.

# CV(Cross-Validation) : KFold(K등분)
gcv = GridSearchCV(estimator=model, param_grid=param_grid, cv=5,
                  n_jobs=-1, verbose=1)

gcv.fit(X_train, y_train)  # 5 x 162 = 810.
print("Best Estimator : ", gcv.best_estimator_) ## 162개의 hyper-parameter 조합 중에, 5개의 서로 다른 데이터로 평가한 평균 성능이 가장 좋은 모델.

In [ ]:
print("Prediction with Best Estimator")
gcv_pred_train = gcv.predict(X_train)
gcv_pred_test = gcv.predict(X_test)

gcv_train_score = evaluation_metric(y_train, gcv_pred_train)
gcv_test_score = evaluation_metric(y_test, gcv_pred_test)

print("Train R2 Score : %.4f" % gcv_train_score)
print("Test R2 Score : %.4f" % gcv_test_score)

In [ ]:
print("Performance Gain") # 이걸로 성능 향상 확인.
print("in train : ", -(train_score - gcv_train_score))
print("in test : ", -(test_score - gcv_test_score))

> optuna를 사용해봅시다 !

In [ ]:
def optimizer(trial, X, y, K):
    # 조절할 hyper-parameter 조합을 적어줍니다.
#     n_estimators = trial.suggest_int('n_estimators', 100, 200)
#     max_depth = trial.suggest_int('max_depth', 4, 8)
#     #max_features = trial.suggest_categorical('max_features', ['sqrt', 'auto', 'log2'])
#     max_features = trial.suggest_float('max_features', 0.5, 0.8)
    
    
#     # 원하는 모델을 지정합니다, optuna는 시간이 오래걸리기 때문에 저는 보통 RF로 일단 테스트를 해본 뒤에 LGBM을 사용합니다.
#     model = RandomForestRegressor(n_estimators=n_estimators,
#                                  max_depth=max_depth,
#                                  max_features=max_features)

    max_depth = trial.suggest_int('max_depth', 5, 20)
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_bin = trial.suggest_int('max_bin', 20, 100)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.01)
    
    model = LGBMRegressor(max_depth=max_depth,
                          n_estimators=n_estimators,
                          max_bin=max_bin,
                          learning_rate=learning_rate)
    
    
    # K-Fold Cross validation을 구현합니다.
    folds = KFold(n_splits=K)
    losses = []
    
    for train_idx, val_idx in folds.split(X, y):
        X_train = X.iloc[train_idx, :]
        y_train = y.iloc[train_idx]
        
        X_val = X.iloc[val_idx, :]
        y_val = y.iloc[val_idx]
        
        model.fit(X_train, y_train)
        preds = model.predict(X_val)
        loss = evaluation_metric(y_val, preds)
        losses.append(loss)
    
    
    # K-Fold의 평균 loss값을 돌려줍니다.
    return np.mean(losses)

In [ ]:
K = 5 # Kfold 수
opt_func = partial(optimizer, X=X_train, y=y_train, K=K)

study = optuna.create_study(study_name='LGBM', direction="maximize") # 최소/최대 어느 방향의 최적값을 구할 건지. # R2 score는 커질수록 좋음.
study.optimize(opt_func, n_trials=1000)

In [ ]:
# optuna가 시도했던 모든 실험 관련 데이터
study.trials_dataframe()

In [ ]:
print("Best Score: %.4f" % study.best_value) # best score 출력
print("Best params: ", study.best_trial.params) # best score일 때의 하이퍼파라미터들

In [ ]:
# 실험 기록 시각화
optuna.visualization.plot_optimization_history(study)

In [ ]:
# hyper-parameter들의 중요도
optuna.visualization.plot_param_importances(study)

### 7. 테스트 및 제출 파일 생성

- prediction using test data


1. test data를 train data와 같은 방식으로 전처리


2. 학습한 모델로 test data 예측


3. submission 파일에 예측 결과를 저장


4. submission 파일 제출

In [ ]:
# 3. PCA
pca_test = pca.transform(test)
X_test = pd.DataFrame(columns=pca_columns, data=pca_test)
X_test

In [ ]:
# 1) optuna에서 찾은 best hyper-parameter 조합으로 학습 후, 예측.  (v)
# 2) optuna가 학습을 할 때, best hyper-parameter를 찾았던 모델을 가져와서 에측.
# model = RandomForestRegressor(n_estimators=study.best_trial.params["n_estimators"],
#                                  max_depth=study.best_trial.params["max_depth"],
#                                  max_features=study.best_trial.params["max_features"])

    
model = LGBMRegressor(max_depth=study.best_trial.params['max_depth'],
                      n_estimators=study.best_trial.params['n_estimators'],
                      max_bin=study.best_trial.params['max_bin'],
                      learning_rate=study.best_trial.params['learning_rate'])


model.fit(X, y) # 전체 데이터로 학습.  ## finalize
preds = model.predict(X_test)
preds

In [ ]:
X_test.shape, preds.shape

In [ ]:
submission = pd.read_csv('../input/mercedes-benz-greener-manufacturing/sample_submission.csv.zip') # submission을 생성합니다.
submission['y'] = preds
submission

In [ ]:
submission.reset_index(drop=True).to_csv("submission.csv", index=False)